# DBSCAN Clustering OPTIMIZED for Large Dataset (8.7M records)

This optimized version uses:
1. Grid-based spatial partitioning
2. Parallel processing with Dask
3. HDBSCAN for faster clustering
4. Progressive sampling strategy

In [ ]:
!pip install hdbscan dask[dataframe] pyarrow fastparquet tqdm folium

## 1. Setup and Imports

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import glob
import os
from collections import defaultdict
import warnings
from multiprocessing import cpu_count
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f"Available CPU cores: {cpu_count()}")

Available CPU cores: 12


## 2. Configuration Parameters

In [3]:
# DBSCAN/HDBSCAN Parameters
EPS_KM = 0.1          # 100 meters
MIN_SAMPLES = 20
USE_HDBSCAN = True    # Set to True for faster clustering

# Grid-based partitioning (speeds up clustering dramatically)
GRID_SIZE_KM = 2.0    # Divide area into 2km x 2km grids
N_CORES = cpu_count() - 1  # Leave one core free

# Data paths
DATA_PATH = '../datos/filtered_amss/sv_12_2023_filtered_amss_part_*.csv'
OUTPUT_DIR = 'results_optimized'

print(f"Configuration:")
print(f"  - EPS: {EPS_KM} km")
print(f"  - Min samples: {MIN_SAMPLES}")
print(f"  - Grid size: {GRID_SIZE_KM} km")
print(f"  - Using HDBSCAN: {USE_HDBSCAN}")
print(f"  - Parallel cores: {N_CORES}")
print(f"  - Output directory: {OUTPUT_DIR}")

Configuration:
  - EPS: 0.1 km
  - Min samples: 20
  - Grid size: 2.0 km
  - Using HDBSCAN: True
  - Parallel cores: 11
  - Output directory: results_optimized


## 3. Load Data with Dask (Memory Efficient)

In [4]:
# Load with Dask for memory efficiency
print("Loading data with Dask (memory-efficient)...")
ddf = dd.read_csv(DATA_PATH, 
                  assume_missing=True,
                  dtype={'device_id': 'object', 
                         'latitude': 'float64', 
                         'longitude': 'float64',
                         'timestamp': 'int64'})

print(f"Total partitions: {ddf.npartitions}")
print(f"Estimated rows: ~{ddf.map_partitions(len).compute().sum():,}")

# Convert to pandas (will fit in memory for AMSS filtered data)
print("\nConverting to pandas...")
df = ddf.compute()

print(f"\n{'='*60}")
print(f"Total records loaded: {len(df):,}")
print(f"Unique devices: {df['device_id'].nunique():,}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"{'='*60}")

Loading data with Dask (memory-efficient)...
Total partitions: 18
Estimated rows: ~8,723,056

Converting to pandas...

Total records loaded: 8,723,056
Unique devices: 11,868
Memory usage: 632.24 MB


## 4. Add Temporal Features

In [5]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['date'] = df['datetime'].dt.date

def classify_time_frame(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    else:
        return 'evening'

df['time_frame'] = df['hour'].apply(classify_time_frame)

print("✓ Temporal features added")

✓ Temporal features added


## 5. SPATIAL GRID PARTITIONING (Key Optimization)

In [6]:
print("Creating spatial grid partitions...")

# Calculate grid coordinates (1 degree ≈ 111 km)
GRID_SIZE_DEG = GRID_SIZE_KM / 111.0

df['grid_lat'] = (df['latitude'] / GRID_SIZE_DEG).astype(int)
df['grid_lon'] = (df['longitude'] / GRID_SIZE_DEG).astype(int)
df['grid_id'] = df['grid_lat'].astype(str) + '_' + df['grid_lon'].astype(str)

n_grids = df['grid_id'].nunique()
grid_sizes = df.groupby('grid_id').size().sort_values(ascending=False)

print(f"\n✓ Created {n_grids} spatial grids")
print(f"  Grid size: {GRID_SIZE_KM} km (~{GRID_SIZE_DEG:.4f} degrees)")
print(f"  Largest grid: {grid_sizes.iloc[0]:,} points")
print(f"  Median grid: {grid_sizes.median():.0f} points")
print(f"  Average grid: {grid_sizes.mean():.0f} points")

print(f"\nTop 10 grids by size:")
for idx, (grid_id, size) in enumerate(grid_sizes.head(10).items(), 1):
    print(f"  {idx}. Grid {grid_id}: {size:,} points")

Creating spatial grid partitions...

✓ Created 255 spatial grids
  Grid size: 2.0 km (~0.0180 degrees)
  Largest grid: 428,527 points
  Median grid: 2533 points
  Average grid: 34208 points

Top 10 grids by size:
  1. Grid 760_-4952: 428,527 points
  2. Grid 760_-4951: 384,835 points
  3. Grid 759_-4952: 353,605 points
  4. Grid 761_-4951: 331,631 points
  5. Grid 760_-4953: 323,209 points
  6. Grid 758_-4953: 312,799 points
  7. Grid 759_-4954: 279,128 points
  8. Grid 760_-4950: 276,186 points
  9. Grid 759_-4955: 274,059 points
  10. Grid 759_-4951: 269,384 points


## 6. PARALLEL CLUSTERING BY GRID

In [10]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.auto import tqdm
import multiprocessing as mp

# Define clustering function at module level for proper pickling
def cluster_single_grid(args):
    """
    Cluster a single grid partition.
    Returns: (grid_id, labels, n_clusters)
    
    This function must be defined at module level for ProcessPoolExecutor.
    """
    grid_id, grid_df, eps_km, min_samples, use_hdbscan = args
    
    if len(grid_df) < min_samples:
        # Not enough points to cluster
        return grid_id, np.full(len(grid_df), -1), 0
    
    coords = grid_df[['latitude', 'longitude']].values
    coords_rad = np.radians(coords)
    
    try:
        if use_hdbscan:
            # HDBSCAN is faster and doesn't need eps
            import hdbscan
            clusterer = hdbscan.HDBSCAN(
                min_cluster_size=min_samples,
                min_samples=min_samples,
                metric='haversine',
                cluster_selection_epsilon=eps_km / 6371.0,  # in radians
                core_dist_n_jobs=1
            )
        else:
            # Standard DBSCAN
            from sklearn.cluster import DBSCAN
            eps_radians = eps_km / 6371.0
            clusterer = DBSCAN(
                eps=eps_radians,
                min_samples=min_samples,
                metric='haversine',
                algorithm='ball_tree',
                n_jobs=1
            )
        
        labels = clusterer.fit_predict(coords_rad)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        return grid_id, labels, n_clusters
    except Exception as e:
        print(f"Error clustering grid {grid_id}: {e}")
        return grid_id, np.full(len(grid_df), -1), 0

print("="*70)
print("CLUSTERING GRIDS IN PARALLEL")
print("="*70)
print(f"Processing {n_grids} grids using {N_CORES} cores...\n")

# Prepare grid data - create list of arguments for each grid
print("Preparing grid data...")
grid_args = []
for grid_id, grid_df in df.groupby('grid_id'):
    grid_args.append((grid_id, grid_df.copy(), EPS_KM, MIN_SAMPLES, USE_HDBSCAN))

print(f"✓ Prepared {len(grid_args)} grids")

# Cluster each grid in parallel
grid_results = {}
total_clusters = 0

# Use spawn method to avoid pickling issues on Windows
if __name__ == '__main__' or 'get_ipython' in dir():
    # For Jupyter/interactive mode, use threading instead of multiprocessing
    print("Using threading mode (safer for Jupyter notebooks)...")
    from concurrent.futures import ThreadPoolExecutor
    
    with ThreadPoolExecutor(max_workers=N_CORES) as executor:
        # Submit all tasks
        futures = {executor.submit(cluster_single_grid, args): args[0] 
                   for args in grid_args}
        
        # Process results as they complete
        with tqdm(total=len(futures), desc="Clustering grids") as pbar:
            for future in as_completed(futures):
                try:
                    grid_id, labels, n_clusters = future.result()
                    grid_results[grid_id] = labels
                    total_clusters += n_clusters
                    pbar.update(1)
                    pbar.set_postfix({'clusters': total_clusters, 'errors': 0})
                except Exception as e:
                    print(f"\\nError processing grid: {e}")
                    pbar.update(1)

print(f"\n✓ Clustering complete!")
print(f"  Total local clusters found: {total_clusters}")
print(f"  Grids processed: {len(grid_results)}/{n_grids}")

CLUSTERING GRIDS IN PARALLEL
Processing 255 grids using 11 cores...

Preparing grid data...
✓ Prepared 255 grids
Using threading mode (safer for Jupyter notebooks)...


Clustering grids: 100%|██████████| 255/255 [33:40<00:00,  7.92s/it, clusters=698, errors=0]   


✓ Clustering complete!
  Total local clusters found: 698
  Grids processed: 255/255


## 7. MERGE GRID RESULTS

In [11]:
print("Merging grid cluster labels...\n")

# Assign cluster labels with grid prefix to avoid conflicts
cluster_labels = np.full(len(df), -1, dtype=int)
cluster_offset = 0

for grid_id in df['grid_id'].unique():
    grid_mask = df['grid_id'] == grid_id
    grid_indices = np.where(grid_mask)[0]
    
    labels = grid_results[grid_id]
    
    # Offset cluster IDs to make them globally unique
    adjusted_labels = labels.copy()
    adjusted_labels[labels != -1] += cluster_offset
    
    cluster_labels[grid_indices] = adjusted_labels
    
    # Update offset for next grid
    max_label = labels.max()
    if max_label >= 0:
        cluster_offset += (max_label + 1)

df['cluster'] = cluster_labels

print("✓ Labels merged!")

Merging grid cluster labels...

✓ Labels merged!


## 8. MERGE BOUNDARY CLUSTERS (Optional)

In [12]:
print("="*70)
print("MERGING BOUNDARY CLUSTERS")
print("="*70)
print("Checking for clusters that span grid boundaries...\n")

from scipy.spatial import cKDTree

# Get cluster centers
clustered_df = df[df['cluster'] != -1].copy()
cluster_centers = clustered_df.groupby('cluster')[['latitude', 'longitude']].mean()

print(f"Found {len(cluster_centers)} initial clusters")

# Build KD-tree for fast neighbor search
coords_rad = np.radians(cluster_centers.values)
tree = cKDTree(coords_rad)

# Find cluster pairs within EPS distance
eps_radians = EPS_KM / 6371.0
pairs = tree.query_pairs(eps_radians)

print(f"Found {len(pairs)} cluster pairs within {EPS_KM} km")

if len(pairs) > 0:
    # Merge clusters using Union-Find
    parent = {i: i for i in cluster_centers.index}
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    
    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[py] = px
    
    cluster_ids = cluster_centers.index.tolist()
    for i, j in pairs:
        union(cluster_ids[i], cluster_ids[j])
    
    # Create cluster mapping
    cluster_map = {c: find(c) for c in cluster_centers.index}
    
    # Apply mapping
    df.loc[df['cluster'] != -1, 'cluster'] = df.loc[df['cluster'] != -1, 'cluster'].map(cluster_map)
    
    # Renumber clusters sequentially
    unique_clusters = sorted(df[df['cluster'] != -1]['cluster'].unique())
    cluster_renumber = {old: new for new, old in enumerate(unique_clusters)}
    df.loc[df['cluster'] != -1, 'cluster'] = df.loc[df['cluster'] != -1, 'cluster'].map(cluster_renumber)
    
    n_final_clusters = len(unique_clusters)
    print(f"\n✓ Merged to {n_final_clusters} final clusters")
else:
    n_final_clusters = len(cluster_centers)
    print("\n✓ No boundary clusters to merge")

MERGING BOUNDARY CLUSTERS
Checking for clusters that span grid boundaries...

Found 698 initial clusters
Found 7 cluster pairs within 0.1 km

✓ Merged to 691 final clusters


## 9. CLUSTERING STATISTICS

In [13]:
n_clusters = len(set(df['cluster'])) - (1 if -1 in df['cluster'].values else 0)
n_noise = (df['cluster'] == -1).sum()
n_clustered = len(df) - n_noise

print(f"{'='*60}")
print(f"FINAL CLUSTERING RESULTS")
print(f"{'='*60}")
print(f"Total clusters found: {n_clusters}")
print(f"Noise points: {n_noise:,} ({100*n_noise/len(df):.2f}%)")
print(f"Clustered points: {n_clustered:,} ({100*n_clustered/len(df):.2f}%)")
print(f"{'='*60}")

# Cluster size statistics
cluster_sizes = df[df['cluster'] != -1]['cluster'].value_counts().sort_index()

print(f"\nCluster Size Statistics:")
print(f"  Mean size: {cluster_sizes.mean():.1f}")
print(f"  Median size: {cluster_sizes.median():.1f}")
print(f"  Largest cluster: {cluster_sizes.max()} points")
print(f"  Smallest cluster: {cluster_sizes.min()} points")

FINAL CLUSTERING RESULTS
Total clusters found: 691
Noise points: 14,530 (0.17%)
Clustered points: 8,708,526 (99.83%)

Cluster Size Statistics:
  Mean size: 12602.8
  Median size: 137.0
  Largest cluster: 427539 points
  Smallest cluster: 20 points


In [14]:
# Save progress
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_file = f'{OUTPUT_DIR}/clustered_data.parquet'

print(f"Saving clustered data to {output_file}...")
df.to_parquet(output_file, index=False, compression='snappy')
print(f"✓ Saved!")

# Continue with the rest of the analysis from the original notebook...

Saving clustered data to results_optimized/clustered_data.parquet...
✓ Saved!


## 10. COMPREHENSIVE CLUSTERING EVALUATION

Now we'll evaluate the clustering quality using multiple metrics and tests.

### 10.1 Standard Clustering Metrics

In [16]:
print("="*70)
print("STANDARD CLUSTERING QUALITY METRICS")
print("="*70)

# Prepare data for evaluation (sample for performance if needed)
SAMPLE_SIZE = min(100000, len(df[df['cluster'] != -1]))
print(f"\nUsing sample of {SAMPLE_SIZE:,} clustered points for evaluation...")

clustered_sample = df[df['cluster'] != -1].sample(SAMPLE_SIZE, random_state=42)
coords_sample = clustered_sample[['latitude', 'longitude']].values
coords_sample_rad = np.radians(coords_sample)
labels_sample = clustered_sample['cluster'].values

# 1. Silhouette Score
print("\n1. Silhouette Score (measures cluster cohesion and separation)")
silhouette_avg = silhouette_score(coords_sample_rad, labels_sample, metric='haversine')
print(f"   Score: {silhouette_avg:.4f}")

if silhouette_avg > 0.7:
    print("   ✓ Excellent clustering quality")
elif silhouette_avg > 0.5:
    print("   ✓ Good clustering quality")
elif silhouette_avg > 0.25:
    print("   ⚠ Acceptable clustering quality")
else:
    print("   ✗ Weak clustering structure")

# 2. Davies-Bouldin Index
print("\n2. Davies-Bouldin Index (lower is better)")
davies_bouldin = davies_bouldin_score(coords_sample_rad, labels_sample)
print(f"   Score: {davies_bouldin:.4f}")

if davies_bouldin < 0.5:
    print("   ✓ Excellent cluster separation")
elif davies_bouldin < 1.0:
    print("   ✓ Good cluster separation")
elif davies_bouldin < 1.5:
    print("   ⚠ Acceptable cluster separation")
else:
    print("   ✗ Poor cluster separation")

# 3. Calinski-Harabasz Index  
print("\n3. Calinski-Harabasz Index (higher is better)")
calinski_harabasz = calinski_harabasz_score(coords_sample_rad, labels_sample)
print(f"   Score: {calinski_harabasz:.2f}")

if calinski_harabasz > 1000:
    print("   ✓ Excellent variance ratio")
elif calinski_harabasz > 500:
    print("   ✓ Good variance ratio")
elif calinski_harabasz > 100:
    print("   ⚠ Acceptable variance ratio")
else:
    print("   ✗ Poor variance ratio")

print("\n" + "="*70)

STANDARD CLUSTERING QUALITY METRICS

Using sample of 100,000 clustered points for evaluation...

1. Silhouette Score (measures cluster cohesion and separation)
   Score: 0.0037
   ✗ Weak clustering structure

2. Davies-Bouldin Index (lower is better)
   Score: 0.6580
   ✓ Good cluster separation

3. Calinski-Harabasz Index (higher is better)
   Score: 20685.13
   ✓ Excellent variance ratio



### 10.2 DBSCAN-Specific Metrics

In [17]:
print("="*70)
print("DBSCAN-SPECIFIC METRICS")
print("="*70)

# 1. Noise Ratio
noise_ratio = n_noise / len(df)
print(f"\n1. Noise Ratio: {noise_ratio:.2%}")
print(f"   Noise points: {n_noise:,}")
print(f"   Clustered points: {n_clustered:,}")

if noise_ratio < 0.10:
    print("   ✓ Excellent - Data is well-grouped")
elif noise_ratio < 0.20:
    print("   ✓ Good - Normal for real-world data")
elif noise_ratio < 0.30:
    print("   ⚠ Acceptable - Consider adjusting eps/min_samples")
else:
    print("   ✗ High noise - Review parameters")

# 2. Cluster Size Distribution
print(f"\n2. Cluster Size Distribution:")
print(f"   Number of clusters: {n_clusters}")
print(f"   Mean cluster size: {cluster_sizes.mean():.1f}")
print(f"   Median cluster size: {cluster_sizes.median():.1f}")
print(f"   Std deviation: {cluster_sizes.std():.1f}")

# Coefficient of Variation
cv = cluster_sizes.std() / cluster_sizes.mean()
print(f"   Coefficient of Variation: {cv:.2f}")

if cv < 0.5:
    print("   ✓ Clusters very uniform")
elif cv < 1.0:
    print("   ✓ Moderate variability (normal)")
else:
    print("   ⚠ High variability - some very large/small clusters")

# Size quantiles
print(f"\n   Size Quantiles:")
for q in [0.25, 0.50, 0.75, 0.90, 0.95, 0.99]:
    val = cluster_sizes.quantile(q)
    print(f"     {int(q*100)}th percentile: {val:.0f} points")

print("\n" + "="*70)

DBSCAN-SPECIFIC METRICS

1. Noise Ratio: 0.17%
   Noise points: 14,530
   Clustered points: 8,708,526
   ✓ Excellent - Data is well-grouped

2. Cluster Size Distribution:
   Number of clusters: 691
   Mean cluster size: 12602.8
   Median cluster size: 137.0
   Std deviation: 48333.4
   Coefficient of Variation: 3.84
   ⚠ High variability - some very large/small clusters

   Size Quantiles:
     25th percentile: 44 points
     50th percentile: 137 points
     75th percentile: 1665 points
     90th percentile: 13045 points
     95th percentile: 78483 points
     99th percentile: 275629 points



### 10.3 Spatial Coherence Analysis

In [18]:
from sklearn.metrics.pairwise import haversine_distances

def calculate_spatial_coherence(df_clustered, n_sample_clusters=30):
    """
    Calculate spatial coherence metrics for clusters.
    Measures how geographically compact each cluster is.
    """
    # Sample top clusters by size for detailed analysis
    top_clusters = cluster_sizes.nlargest(n_sample_clusters).index
    
    coherence_data = []
    
    for cluster_id in tqdm(top_clusters, desc="Analyzing spatial coherence"):
        cluster_data = df_clustered[df_clustered['cluster'] == cluster_id]
        
        if len(cluster_data) < 2:
            continue
            
        # Get coordinates
        coords = cluster_data[['latitude', 'longitude']].values
        coords_rad = np.radians(coords)
        
        # Calculate center
        center = coords_rad.mean(axis=0).reshape(1, -1)
        
        # Distances from center (in km)
        distances = haversine_distances(coords_rad, center)[:, 0] * 6371
        
        coherence_data.append({
            'cluster_id': cluster_id,
            'size': len(cluster_data),
            'avg_radius_km': distances.mean(),
            'max_radius_km': distances.max(),
            'std_radius_km': distances.std(),
            'compactness': distances.mean() / distances.max() if distances.max() > 0 else 1.0
        })
    
    return pd.DataFrame(coherence_data)

print("="*70)
print("SPATIAL COHERENCE ANALYSIS")
print("="*70)
print(f"\nAnalyzing top 30 clusters by size...")

coherence_df = calculate_spatial_coherence(df[df['cluster'] != -1])

print(f"\n✓ Analysis complete!")
print(f"\nGlobal Statistics:")
print(f"  Average cluster radius: {coherence_df['avg_radius_km'].mean():.4f} km")
print(f"  Median cluster radius: {coherence_df['avg_radius_km'].median():.4f} km")
print(f"  Max cluster radius: {coherence_df['max_radius_km'].max():.4f} km")
print(f"  Average compactness: {coherence_df['compactness'].mean():.4f}")

# Interpretation
avg_radius = coherence_df['avg_radius_km'].mean()
if avg_radius < EPS_KM:
    print(f"\n✓ Excellent - Clusters very compact (avg radius < eps)")
elif avg_radius < 2 * EPS_KM:
    print(f"\n✓ Good - Clusters reasonably compact (avg radius ≈ eps)")
else:
    print(f"\n⚠ Warning - Clusters dispersed (avg radius > 2*eps)")

print(f"\nTop 10 most compact clusters:")
print(coherence_df.nsmallest(10, 'avg_radius_km')[['cluster_id', 'size', 'avg_radius_km', 'compactness']])

print(f"\nTop 10 largest (by radius) clusters:")
print(coherence_df.nlargest(10, 'avg_radius_km')[['cluster_id', 'size', 'avg_radius_km', 'compactness']])

print("\n" + "="*70)

SPATIAL COHERENCE ANALYSIS

Analyzing top 30 clusters by size...


Analyzing spatial coherence: 100%|██████████| 30/30 [00:02<00:00, 12.80it/s]



✓ Analysis complete!

Global Statistics:
  Average cluster radius: 0.6833 km
  Median cluster radius: 0.6821 km
  Max cluster radius: 1.7492 km
  Average compactness: 0.4695

⚠ Warning - Clusters dispersed (avg radius > 2*eps)

Top 10 most compact clusters:
    cluster_id    size  avg_radius_km  compactness
24          18  131705       0.545432     0.403698
17          13  172447       0.581840     0.418370
15           5  186739       0.584830     0.416872
23         357  131964       0.603696     0.381585
16         152  175786       0.604298     0.372921
28          22  109255       0.607906     0.419609
19         108  147628       0.623476     0.356433
4           31  314604       0.635846     0.438476
5           16  312675       0.639278     0.435962
27          24  111525       0.647972     0.415106

Top 10 largest (by radius) clusters:
    cluster_id    size  avg_radius_km  compactness
13          26  231803       0.835212     0.590150
21         106  143426       0.799269   

## 11. INTERACTIVE MAP VISUALIZATION WITH FOLIUM

Visualize clusters on an interactive map

In [19]:
import folium
from folium.plugins import MarkerCluster, HeatMap
import random

print("Creating interactive maps...")

# Calculate map center
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

print(f"Map center: ({center_lat:.6f}, {center_lon:.6f})")

Creating interactive maps...
Map center: (13.700023, -89.219494)


### 11.1 Top Clusters Overview Map

In [20]:
# Map 1: Top 20 clusters with centroids
print("Creating Map 1: Top 20 Clusters Overview...")

m1 = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Get top 20 clusters
top_20_clusters = cluster_sizes.nlargest(20).index

# Generate colors for clusters
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
          'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 
          'darkpurple', 'pink', 'lightblue', 'lightgreen', 
          'gray', 'black', 'lightgray']

# Sample points from each cluster (100 points per cluster max)
POINTS_PER_CLUSTER = 100

for idx, cluster_id in enumerate(top_20_clusters):
    cluster_data = df[df['cluster'] == cluster_id]
    
    # Sample points
    if len(cluster_data) > POINTS_PER_CLUSTER:
        cluster_sample = cluster_data.sample(POINTS_PER_CLUSTER, random_state=42)
    else:
        cluster_sample = cluster_data
    
    # Calculate cluster center
    center_lat_cluster = cluster_data['latitude'].mean()
    center_lon_cluster = cluster_data['longitude'].mean()
    cluster_size = len(cluster_data)
    
    color = colors[idx % len(colors)]
    
    # Add cluster centroid marker
    folium.Marker(
        location=[center_lat_cluster, center_lon_cluster],
        popup=f"""
        <b>Cluster {cluster_id}</b><br>
        Size: {cluster_size:,} points<br>
        Rank: #{idx+1}<br>
        Center: ({center_lat_cluster:.6f}, {center_lon_cluster:.6f})
        """,
        tooltip=f"Cluster {cluster_id} ({cluster_size:,} points)",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(m1)
    
    # Add circle showing approximate cluster area
    folium.Circle(
        location=[center_lat_cluster, center_lon_cluster],
        radius=EPS_KM * 1000,  # Convert km to meters
        popup=f"Cluster {cluster_id} (EPS radius)",
        color=color,
        fill=True,
        fillOpacity=0.1
    ).add_to(m1)

# Save map
map1_file = f'{OUTPUT_DIR}/map_top20_clusters.html'
m1.save(map1_file)
print(f"✓ Saved: {map1_file}")

# Display in notebook
print("\\nDisplaying map (may take a moment to render)...")
m1

Creating Map 1: Top 20 Clusters Overview...
✓ Saved: results_optimized/map_top20_clusters.html
\nDisplaying map (may take a moment to render)...


### 11.2 Activity Heatmap

In [ ]:
# Map 2: Heatmap of all activity
print("Creating Map 2: Activity Heatmap...")

m2 = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='CartoDB positron'
)

# Sample data for heatmap (max 10,000 points for performance)
HEATMAP_SAMPLE_SIZE = 10000
heatmap_sample = df[df['cluster'] != -1].sample(min(HEATMAP_SAMPLE_SIZE, len(df[df['cluster'] != -1])), random_state=42)

# Prepare heat data
heat_data = [[row['latitude'], row['longitude']] for _, row in heatmap_sample.iterrows()]

# Add heatmap layer
HeatMap(
    heat_data,
    min_opacity=0.3,
    max_zoom=18,
    radius=15,
    blur=20,
    gradient={
        0.0: 'blue',
        0.3: 'lime',
        0.5: 'yellow',
        0.7: 'orange',
        1.0: 'red'
    }
).add_to(m2)

# Save map
map2_file = f'{OUTPUT_DIR}/map_activity_heatmap.html'
m2.save(map2_file)
print(f"✓ Saved: {map2_file}")

print("\\nDisplaying heatmap...")
m2

### 11.3 Cluster Centers with MarkerCluster

In [ ]:
# Map 3: All cluster centers with MarkerCluster
print("Creating Map 3: All Cluster Centers...")

m3 = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Create marker cluster
marker_cluster = MarkerCluster().add_to(m3)

# Calculate centers for all clusters
cluster_centers_df = df[df['cluster'] != -1].groupby('cluster').agg({
    'latitude': 'mean',
    'longitude': 'mean',
    'device_id': 'count'
}).rename(columns={'device_id': 'size'}).reset_index()

# Add markers for each cluster center
for _, row in cluster_centers_df.iterrows():
    # Size category for icon color
    size = row['size']
    if size > 10000:
        color = 'red'
        category = 'Very Large'
    elif size > 1000:
        color = 'orange'
        category = 'Large'
    elif size > 100:
        color = 'blue'
        category = 'Medium'
    else:
        color = 'green'
        category = 'Small'
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"""
        <div style='width: 200px'>
            <h4>Cluster {row['cluster']}</h4>
            <b>Size:</b> {row['size']:,} points<br>
            <b>Category:</b> {category}<br>
            <b>Center:</b> ({row['latitude']:.6f}, {row['longitude']:.6f})<br>
        </div>
        """,
        tooltip=f"Cluster {row['cluster']} - {row['size']:,} points",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

# Add legend
legend_html = '''
<div style="position: fixed; 
    bottom: 50px; right: 50px; width: 180px; height: 150px; 
    background-color: white; border:2px solid grey; z-index:9999; 
    font-size:14px; padding: 10px">
    <p><b>Cluster Sizes</b></p>
    <p><i class="fa fa-map-marker fa-2x" style="color:red"></i> Very Large (>10K)</p>
    <p><i class="fa fa-map-marker fa-2x" style="color:orange"></i> Large (>1K)</p>
    <p><i class="fa fa-map-marker fa-2x" style="color:blue"></i> Medium (>100)</p>
    <p><i class="fa fa-map-marker fa-2x" style="color:green"></i> Small (<100)</p>
</div>
'''
m3.get_root().html.add_child(folium.Element(legend_html))

# Save map
map3_file = f'{OUTPUT_DIR}/map_all_cluster_centers.html'
m3.save(map3_file)
print(f"✓ Saved: {map3_file}")

print(f"\\nCreated {len(cluster_centers_df)} cluster center markers")
print("\\nDisplaying map (zoom in to see individual clusters)...")
m3

## 12. FINAL SUMMARY AND EXPORT

In [ ]:
# Save comprehensive results
print("="*70)
print("SAVING RESULTS")
print("="*70)

# Save clustered data
output_file = f'{OUTPUT_DIR}/clustered_data.parquet'
print(f"\\nSaving clustered data to {output_file}...")
df.to_parquet(output_file, index=False, compression='snappy')
print(f"✓ Saved!")

# Save cluster statistics
cluster_stats_df = cluster_centers_df.merge(
    coherence_df[['cluster_id', 'avg_radius_km', 'compactness']],
    left_on='cluster',
    right_on='cluster_id',
    how='left'
)
stats_file = f'{OUTPUT_DIR}/cluster_statistics.csv'
cluster_stats_df.to_csv(stats_file, index=False)
print(f"✓ Cluster statistics saved: {stats_file}")

# Save evaluation metrics
metrics = {
    'n_clusters': n_clusters,
    'n_noise': n_noise,
    'n_clustered': n_clustered,
    'noise_ratio': noise_ratio,
    'silhouette_score': silhouette_avg,
    'davies_bouldin_index': davies_bouldin,
    'calinski_harabasz_index': calinski_harabasz,
    'avg_cluster_size': cluster_sizes.mean(),
    'median_cluster_size': cluster_sizes.median(),
    'avg_cluster_radius_km': coherence_df['avg_radius_km'].mean(),
    'avg_compactness': coherence_df['compactness'].mean(),
    'eps_km': EPS_KM,
    'min_samples': MIN_SAMPLES,
    'grid_size_km': GRID_SIZE_KM,
    'use_hdbscan': USE_HDBSCAN
}

metrics_file = f'{OUTPUT_DIR}/evaluation_metrics.pkl'
with open(metrics_file, 'wb') as f:
    pickle.dump(metrics, f)
print(f"✓ Evaluation metrics saved: {metrics_file}")

print("\\n" + "="*70)
print("CLUSTERING COMPLETE!")
print("="*70)
print(f"\\nFiles generated:")
print(f"  Data:")
print(f"    - {output_file}")
print(f"    - {stats_file}")
print(f"    - {metrics_file}")
print(f"  Maps:")
print(f"    - {map1_file}")
print(f"    - {map2_file}")
print(f"    - {map3_file}")
print("\\n" + "="*70)

In [ ]:
# Print comprehensive summary
print("\\n" + "="*70)
print("COMPREHENSIVE EVALUATION SUMMARY")
print("="*70)

print(f"\\n📊 DATASET:")
print(f"  Total points: {len(df):,}")
print(f"  Unique devices: {df['device_id'].nunique():,}")

print(f"\\n🎯 CLUSTERING RESULTS:")
print(f"  Clusters found: {n_clusters}")
print(f"  Clustered points: {n_clustered:,} ({100*n_clustered/len(df):.2f}%)")
print(f"  Noise points: {n_noise:,} ({100*noise_ratio:.2f}%)")

print(f"\\n📈 QUALITY METRICS:")
print(f"  Silhouette Score: {silhouette_avg:.4f}", end="")
if silhouette_avg > 0.5:
    print(" ✓ Good")
elif silhouette_avg > 0.25:
    print(" ⚠ Acceptable")
else:
    print(" ✗ Weak")

print(f"  Davies-Bouldin: {davies_bouldin:.4f}", end="")
if davies_bouldin < 1.0:
    print(" ✓ Good")
elif davies_bouldin < 1.5:
    print(" ⚠ Acceptable")
else:
    print(" ✗ Poor")

print(f"  Calinski-Harabasz: {calinski_harabasz:.2f}", end="")
if calinski_harabasz > 500:
    print(" ✓ Good")
elif calinski_harabasz > 100:
    print(" ⚠ Acceptable")
else:
    print(" ✗ Poor")

print(f"\\n📏 SIZE DISTRIBUTION:")
print(f"  Mean: {cluster_sizes.mean():.1f}")
print(f"  Median: {cluster_sizes.median():.1f}")
print(f"  Min: {cluster_sizes.min()}")
print(f"  Max: {cluster_sizes.max():,}")
print(f"  CV: {cv:.2f}", end="")
if cv < 1.0:
    print(" ✓ Moderate variability")
else:
    print(" ⚠ High variability")

print(f"\\n🗺️  SPATIAL COHERENCE:")
print(f"  Avg radius: {coherence_df['avg_radius_km'].mean():.4f} km", end="")
if coherence_df['avg_radius_km'].mean() < EPS_KM:
    print(" ✓ Very compact")
elif coherence_df['avg_radius_km'].mean() < 2*EPS_KM:
    print(" ✓ Compact")
else:
    print(" ⚠ Dispersed")

print(f"  Avg compactness: {coherence_df['compactness'].mean():.4f}")

print(f"\\n⚙️  PARAMETERS USED:")
print(f"  EPS: {EPS_KM} km")
print(f"  Min samples: {MIN_SAMPLES}")
print(f"  Grid size: {GRID_SIZE_KM} km")
print(f"  Algorithm: {'HDBSCAN' if USE_HDBSCAN else 'DBSCAN'}")

print(f"\\n🗺️  MAPS GENERATED:")
print(f"  1. Top 20 Clusters Overview")
print(f"  2. Activity Heatmap")
print(f"  3. All Cluster Centers (MarkerCluster)")

print("\\n" + "="*70)
print("✓ ANALYSIS COMPLETE!")
print("="*70)